## Coursera Capstone Project

## The Battle of Two Cities

### The cities here in consideration are Delhi and Bombay

Mumbai and Delhi are the two most important metro cities in India. There has been a war for supremacy in terms of quality of life, jobs, education, entertainment and recreational facilities that these cities have to offer to its residents. I  attemp to analyze the neighborhoods in each of these two cities and try to understand what is popular in them and what they have to offer to someone who is contemplating to make a choice on seeking a life in either of the metro cities of India.


### The Business Problem 

The business problem in this study assumes that people who would be interested in this study are those who would like to create a projection of potential life and activities in these metro city neighborhoods if the subject moves to live in one of them. The decision to choose one over the other would depend on popular venues in the neighborhoods in each of these metro cities.

#### Importing all the necessary Libraries 

In [2]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from geopy.extra.rate_limiter import RateLimiter

About the data
It is a data provided by the idian government.
link:https://data.gov.in/catalog/all-india-pincode-directory?filters%5Bfield_catalog_reference%5D=85840&format=json&offset=0&limit=6&sort%5Bcreated%5D=desc
The data published by the government is on postal codes for all India would help me in the project.

In [3]:
df=pd.read_csv('all_india_PO_list_without_APS_offices_ver2_lat_long.csv')
df.head()

,officename,pincode,officeType,Deliverystatus,divisionname,regionname,circlename,Taluk,Districtname,statename,Telephone,Related Suboffice,Related Headoffice,longitude,latitude
0,Achalapur B.O,504273,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Asifabad,Adilabad,TELANGANA,NaN,Rechini S.O,Mancherial H.O,NaN,NaN
1,Ada B.O,504293,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Asifabad,Adilabad,TELANGANA,NaN,Asifabad S.O,Mancherial H.O,NaN,NaN
2,Adegaon B.O,504307,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Boath,Adilabad,TELANGANA,NaN,Echoda S.O,Adilabad H.O,NaN,NaN
3,Adilabad Collectorate S.O,504001,S.O,Non-Delivery,Adilabad,Hyderabad,Andhra Pradesh,Adilabad,Adilabad,TELANGANA,08732-226703,NaN,Adilabad H.O,NaN,NaN
4,Adilabad H.O,504001,H.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Adilabad,Adilabad,TELANGANA,08732-226738,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['officename', 'pincode', 'officeType', 'Deliverystatus', 'divisionname',
       'regionname', 'circlename', 'Taluk', 'Districtname', 'statename',
       'Telephone', 'Related Suboffice', 'Related Headoffice', 'longitude',
       'latitude'],
      dtype='object')

In [4]:
df.shape

(154797, 15)

In [5]:
df['regionname'].value_counts()

Muzaffarpur                5150
Hyderabad                  4674
South Karnataka            4594
North Karnataka            4487
Vijayawada                 4477
Jodhpur                    4457
Bhopal HQ                  4201
South Bengal               3955
Patna HQ                   3913
Aurangabad                 3882
Gorakhpur                  3700
Vadodara                   3686
Kurnool                    3680
Bhubaneswar HQ             3531
Tiruchy                    3505
Bareilly                   3504
Ajmer                      3502
Calcutta                   3183
Madurai                    3164
Raipur                     3150
Ranchi                     3097
Coimbatore                 3047
Agra                       3043
Rajkot                     3029
Lucknow  HQ                2992
Nagpur                     2931
Indore                     2874
Visakhapatnam              2781
Shimla HQ                  2781
Dehradun                   2715
Ambala  HQ                 2689
Chandiga

### from the loaded dataset we create two dataframes
    1. One for mumbai
    2. one for delhi
    First we make for mumbai

In [6]:
df_mumb=df[df['regionname']=='Mumbai']

In [7]:
df_mumb.head()

,officename,pincode,officeType,Deliverystatus,divisionname,regionname,circlename,Taluk,Districtname,statename,Telephone,Related Suboffice,Related Headoffice,longitude,latitude
81395,Antop Hill S.O,400037,S.O,Delivery,Mumbai East,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-24120290,NaN,Dadar H.O,NaN,NaN
81396,B P T Colony S.O,400037,S.O,Non-Delivery,Mumbai East,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-4100525,NaN,Dadar H.O,NaN,NaN
81397,B.P.Lane S.O,400003,S.O,Non-Delivery,Mumbai East,Mumbai,Maharashtra,NaN,Mumbai,MAHARASHTRA,022-23421653,NaN,Chinchbunder H.O,NaN,NaN
81398,BEST STaff Quarters S.O,400012,S.O,Non-Delivery,Mumbai East,Mumbai,Maharashtra,NaN,Mumbai,MAHARASHTRA,022-24180776,NaN,Chinchbunder H.O,NaN,NaN
81399,C G S Colony S.O,400037,S.O,Non-Delivery,Mumbai East,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-24097933,NaN,Dadar H.O,NaN,NaN


We add a new column to the dataset namely Address which will be ',' separated and will have 3 features which help us to determine the latitude adn longitude of the address

In [8]:
df_mumb['Address']=df_mumb['officename']+','+df_mumb['regionname']+','+df_mumb['statename']

c:\users\addy saish\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
df_mumb.shape

(1123, 16)

In [10]:
df_mumb.head()

,officename,pincode,officeType,Deliverystatus,divisionname,regionname,circlename,Taluk,Districtname,statename,Telephone,Related Suboffice,Related Headoffice,longitude,latitude,Address
81395,Antop Hill S.O,400037,S.O,Delivery,Mumbai East,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-24120290,NaN,Dadar H.O,NaN,NaN,"Antop Hill S.O,Mumbai,MAHARASHTRA"
81396,B P T Colony S.O,400037,S.O,Non-Delivery,Mumbai East,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-4100525,NaN,Dadar H.O,NaN,NaN,"B P T Colony S.O,Mumbai,MAHARASHTRA"
81397,B.P.Lane S.O,400003,S.O,Non-Delivery,Mumbai East,Mumbai,Maharashtra,NaN,Mumbai,MAHARASHTRA,022-23421653,NaN,Chinchbunder H.O,NaN,NaN,"B.P.Lane S.O,Mumbai,MAHARASHTRA"
81398,BEST STaff Quarters S.O,400012,S.O,Non-Delivery,Mumbai East,Mumbai,Maharashtra,NaN,Mumbai,MAHARASHTRA,022-24180776,NaN,Chinchbunder H.O,NaN,NaN,"BEST STaff Quarters S.O,Mumbai,MAHARASHTRA"
81399,C G S Colony S.O,400037,S.O,Non-Delivery,Mumbai East,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-24097933,NaN,Dadar H.O,NaN,NaN,"C G S Colony S.O,Mumbai,MAHARASHTRA"


Creating the dataframe having only values related to delhi

In [11]:
df_del=df[df['regionname']=='Delhi']

In [12]:
df_del.head()

,officename,pincode,officeType,Deliverystatus,divisionname,regionname,circlename,Taluk,Districtname,statename,Telephone,Related Suboffice,Related Headoffice,longitude,latitude
32383,Anand Vihar S.O,110092,S.O,Non-Delivery,Delhi East,Delhi,Delhi,NaN,East Delhi,DELHI,011-22157472,NaN,Krishna Nagar H.O,NaN,NaN
32384,Azad Nagar S.O (East Delhi),110051,S.O,Non-Delivery,Delhi East,Delhi,Delhi,NaN,East Delhi,DELHI,011-22093521,NaN,Krishna Nagar H.O,NaN,NaN
32385,Babarpur S.O (North East Delhi),110032,S.O,Non-Delivery,Delhi East,Delhi,Delhi,NaN,North East Delhi,DELHI,011-22829634,NaN,Jhilmil H.O,NaN,NaN
32386,Badarpur Khadar B.O,110090,B.O,Delivery,Delhi East,Delhi,Delhi,East Delhi,East Delhi,DELHI,NaN,Karawal Nagar S.O,Jhilmil H.O,NaN,NaN
32387,Balbir Nagar S.O,110032,S.O,Non-Delivery,Delhi East,Delhi,Delhi,NaN,East Delhi,DELHI,011-22320223,NaN,Jhilmil H.O,NaN,NaN


In [13]:
df_del.shape

(545, 15)

We add a new column to the dataset namely Address which will be ',' separated and will have 3 features which help us to determine the latitude adn longitude of the address

In [14]:
df_del['Address']=df_del['officename']+','+df_del['regionname']+','+df_del['statename']

c:\users\addy saish\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
df_del.head()

,officename,pincode,officeType,Deliverystatus,divisionname,regionname,circlename,Taluk,Districtname,statename,Telephone,Related Suboffice,Related Headoffice,longitude,latitude,Address
32383,Anand Vihar S.O,110092,S.O,Non-Delivery,Delhi East,Delhi,Delhi,NaN,East Delhi,DELHI,011-22157472,NaN,Krishna Nagar H.O,NaN,NaN,"Anand Vihar S.O,Delhi,DELHI"
32384,Azad Nagar S.O (East Delhi),110051,S.O,Non-Delivery,Delhi East,Delhi,Delhi,NaN,East Delhi,DELHI,011-22093521,NaN,Krishna Nagar H.O,NaN,NaN,"Azad Nagar S.O (East Delhi),Delhi,DELHI"
32385,Babarpur S.O (North East Delhi),110032,S.O,Non-Delivery,Delhi East,Delhi,Delhi,NaN,North East Delhi,DELHI,011-22829634,NaN,Jhilmil H.O,NaN,NaN,"Babarpur S.O (North East Delhi),Delhi,DELHI"
32386,Badarpur Khadar B.O,110090,B.O,Delivery,Delhi East,Delhi,Delhi,East Delhi,East Delhi,DELHI,NaN,Karawal Nagar S.O,Jhilmil H.O,NaN,NaN,"Badarpur Khadar B.O,Delhi,DELHI"
32387,Balbir Nagar S.O,110032,S.O,Non-Delivery,Delhi East,Delhi,Delhi,NaN,East Delhi,DELHI,011-22320223,NaN,Jhilmil H.O,NaN,NaN,"Balbir Nagar S.O,Delhi,DELHI"


In [16]:
df_del.shape

(545, 16)

Dropping all the unnecessary columns

In [17]:
df_mumb.drop(['officeType','Deliverystatus','divisionname','Taluk','Districtname','statename','Telephone','circlename','Related Suboffice','Related Headoffice'],axis=1,inplace=True)

c:\users\addy saish\python\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [18]:
df_mumb.head()

,officename,pincode,regionname,longitude,latitude,Address
81395,Antop Hill S.O,400037,Mumbai,NaN,NaN,"Antop Hill S.O,Mumbai,MAHARASHTRA"
81396,B P T Colony S.O,400037,Mumbai,NaN,NaN,"B P T Colony S.O,Mumbai,MAHARASHTRA"
81397,B.P.Lane S.O,400003,Mumbai,NaN,NaN,"B.P.Lane S.O,Mumbai,MAHARASHTRA"
81398,BEST STaff Quarters S.O,400012,Mumbai,NaN,NaN,"BEST STaff Quarters S.O,Mumbai,MAHARASHTRA"
81399,C G S Colony S.O,400037,Mumbai,NaN,NaN,"C G S Colony S.O,Mumbai,MAHARASHTRA"


Renaming the officename column to Neighborhood column

In [19]:
df_mumb.rename(columns={'officename':'Neighborhood'},inplace=True)

c:\users\addy saish\python\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [20]:
df_mumb.head()

,Neighborhood,pincode,regionname,longitude,latitude,Address
81395,Antop Hill S.O,400037,Mumbai,NaN,NaN,"Antop Hill S.O,Mumbai,MAHARASHTRA"
81396,B P T Colony S.O,400037,Mumbai,NaN,NaN,"B P T Colony S.O,Mumbai,MAHARASHTRA"
81397,B.P.Lane S.O,400003,Mumbai,NaN,NaN,"B.P.Lane S.O,Mumbai,MAHARASHTRA"
81398,BEST STaff Quarters S.O,400012,Mumbai,NaN,NaN,"BEST STaff Quarters S.O,Mumbai,MAHARASHTRA"
81399,C G S Colony S.O,400037,Mumbai,NaN,NaN,"C G S Colony S.O,Mumbai,MAHARASHTRA"


In [21]:
df_del.drop(['officeType','Deliverystatus','divisionname','Taluk','Districtname','statename','Telephone','circlename','Related Suboffice','Related Headoffice'],axis=1,inplace=True)

In [22]:
df_del.rename(columns={'officename':'Neighborhood'},inplace=True)

In [23]:
df_del.head()

,Neighborhood,pincode,regionname,longitude,latitude,Address
32383,Anand Vihar S.O,110092,Delhi,NaN,NaN,"Anand Vihar S.O,Delhi,DELHI"
32384,Azad Nagar S.O (East Delhi),110051,Delhi,NaN,NaN,"Azad Nagar S.O (East Delhi),Delhi,DELHI"
32385,Babarpur S.O (North East Delhi),110032,Delhi,NaN,NaN,"Babarpur S.O (North East Delhi),Delhi,DELHI"
32386,Badarpur Khadar B.O,110090,Delhi,NaN,NaN,"Badarpur Khadar B.O,Delhi,DELHI"
32387,Balbir Nagar S.O,110032,Delhi,NaN,NaN,"Balbir Nagar S.O,Delhi,DELHI"


In [24]:
df_mumb.drop(['longitude','latitude'],axis=1,inplace=True)

In [25]:
df_mumb.head()

,Neighborhood,pincode,regionname,Address
81395,Antop Hill S.O,400037,Mumbai,"Antop Hill S.O,Mumbai,MAHARASHTRA"
81396,B P T Colony S.O,400037,Mumbai,"B P T Colony S.O,Mumbai,MAHARASHTRA"
81397,B.P.Lane S.O,400003,Mumbai,"B.P.Lane S.O,Mumbai,MAHARASHTRA"
81398,BEST STaff Quarters S.O,400012,Mumbai,"BEST STaff Quarters S.O,Mumbai,MAHARASHTRA"
81399,C G S Colony S.O,400037,Mumbai,"C G S Colony S.O,Mumbai,MAHARASHTRA"


In [26]:
df_del.drop(['longitude','latitude'],axis=1,inplace=True)

In [27]:
df_del.head()

,Neighborhood,pincode,regionname,Address
32383,Anand Vihar S.O,110092,Delhi,"Anand Vihar S.O,Delhi,DELHI"
32384,Azad Nagar S.O (East Delhi),110051,Delhi,"Azad Nagar S.O (East Delhi),Delhi,DELHI"
32385,Babarpur S.O (North East Delhi),110032,Delhi,"Babarpur S.O (North East Delhi),Delhi,DELHI"
32386,Badarpur Khadar B.O,110090,Delhi,"Badarpur Khadar B.O,Delhi,DELHI"
32387,Balbir Nagar S.O,110032,Delhi,"Balbir Nagar S.O,Delhi,DELHI"


Here we using nominatim to locate the Coordinates of the address column in the dataframe. Rate limiter is used to efficiently search the coordinates as lots of coornites are to be found.

In [35]:
nom=Nominatim(user_agent='myGeocoder')
geocode = RateLimiter(nom.geocode, min_delay_seconds=1)
df_del['Coordinates']=df_del['Address'].apply(geocode)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Dilshad Garden S.O,Delhi,DELHI',), **{}).
Traceback (most recent call last):
  File "c:\users\addy saish\python\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "c:\users\addy saish\python\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1016, in _send_output
    self.send(msg)
  File "c:\users\addy saish\python\lib\http\client.py", line 956, in send
    self.connect()
  File "c:\users\addy saish\python\lib\http\client.py", line 1384, in connect
    super().

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Ghoga B.O,Delhi,DELHI',), **{}).
Traceback (most recent call last):
  File "c:\users\addy saish\python\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "c:\users\addy saish\python\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1016, in _send_output
    self.send(msg)
  File "c:\users\addy saish\python\lib\http\client.py", line 956, in send
    self.connect()
  File "c:\users\addy saish\python\lib\http\client.py", line 1392, in connect
    server_hostname=s

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Sarita Vihar S.O,Delhi,DELHI',), **{}).
Traceback (most recent call last):
  File "c:\users\addy saish\python\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "c:\users\addy saish\python\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1016, in _send_output
    self.send(msg)
  File "c:\users\addy saish\python\lib\http\client.py", line 956, in send
    self.connect()
  File "c:\users\addy saish\python\lib\http\client.py", line 1392, in connect
    server_hos

c:\users\addy saish\python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
df_del.head()

,Neighborhood,pincode,regionname,Address,Coordinates
32383,Anand Vihar S.O,110092,Delhi,"Anand Vihar S.O,Delhi,DELHI","(Anand Vihar, Vivek Vihar Tehsil, Shahdara, De..."
32384,Azad Nagar S.O (East Delhi),110051,Delhi,"Azad Nagar S.O (East Delhi),Delhi,DELHI",None
32385,Babarpur S.O (North East Delhi),110032,Delhi,"Babarpur S.O (North East Delhi),Delhi,DELHI",None
32386,Badarpur Khadar B.O,110090,Delhi,"Badarpur Khadar B.O,Delhi,DELHI",None
32387,Balbir Nagar S.O,110032,Delhi,"Balbir Nagar S.O,Delhi,DELHI","(Balbir Nagar, Babarpur, Shahdara Tehsil, Shah..."


here we adding a new column for latitude to the dataframe by defining a lambda function which will return the latitude from the coordinates column 

In [37]:
df_del['latitude']=df_del['Coordinates'].apply(lambda x: x.latitude if x!= None else None)

c:\users\addy saish\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
df_del['longitude']=df_del['Coordinates'].apply(lambda x: x.longitude if x!= None else None)

c:\users\addy saish\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
df_del.head()

,Neighborhood,pincode,regionname,Address,Coordinates,latitude,longitude
32383,Anand Vihar S.O,110092,Delhi,"Anand Vihar S.O,Delhi,DELHI","(Anand Vihar, Vivek Vihar Tehsil, Shahdara, De...",28.641115,77.312502
32384,Azad Nagar S.O (East Delhi),110051,Delhi,"Azad Nagar S.O (East Delhi),Delhi,DELHI",None,NaN,NaN
32385,Babarpur S.O (North East Delhi),110032,Delhi,"Babarpur S.O (North East Delhi),Delhi,DELHI",None,NaN,NaN
32386,Badarpur Khadar B.O,110090,Delhi,"Badarpur Khadar B.O,Delhi,DELHI",None,NaN,NaN
32387,Balbir Nagar S.O,110032,Delhi,"Balbir Nagar S.O,Delhi,DELHI","(Balbir Nagar, Babarpur, Shahdara Tehsil, Shah...",28.683790,77.290754


Drop all the rows that reurned NaN in the latitude and longitude column

In [40]:
df_del=df_del.dropna()

In [43]:
df_del.head()

,Neighborhood,pincode,regionname,Address,Coordinates,latitude,longitude
32383,Anand Vihar S.O,110092,Delhi,"Anand Vihar S.O,Delhi,DELHI","(Anand Vihar, Vivek Vihar Tehsil, Shahdara, De...",28.641115,77.312502
32387,Balbir Nagar S.O,110032,Delhi,"Balbir Nagar S.O,Delhi,DELHI","(Balbir Nagar, Babarpur, Shahdara Tehsil, Shah...",28.683790,77.290754
32389,Bhola Nath Nagar S.O,110032,Delhi,"Bhola Nath Nagar S.O,Delhi,DELHI","(Bhola Nath Nagar, Vivek Vihar Tehsil, Shahdar...",28.669127,77.285241
32393,Dilshad Garden S.O,110095,Delhi,"Dilshad Garden S.O,Delhi,DELHI","(Dilshad Garden, GT Road, Dilshad Garden, Vive...",28.675826,77.321516
32399,Geeta Colony S.O,110031,Delhi,"Geeta Colony S.O,Delhi,DELHI","(Geeta Colony, Preet Vihar Tehsil, East Delhi,...",28.650101,77.275921


In [41]:
df_del.shape

(204, 7)

In [46]:
geocode = RateLimiter(nom.geocode, min_delay_seconds=3)
df_mumb['Coordinates']=df_mumb['Address'].apply(geocode)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Antop Hill S.O,Mumbai,MAHARASHTRA',), **{}).
Traceback (most recent call last):
  File "c:\users\addy saish\python\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "c:\users\addy saish\python\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1016, in _send_output
    self.send(msg)
  File "c:\users\addy saish\python\lib\http\client.py", line 956, in send
    self.connect()
  File "c:\users\addy saish\python\lib\http\client.py", line 1392, in connect
    serve

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Kidwai Nagar S.O (Mumbai),Mumbai,MAHARASHTRA',), **{}).
Traceback (most recent call last):
  File "c:\users\addy saish\python\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "c:\users\addy saish\python\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1016, in _send_output
    self.send(msg)
  File "c:\users\addy saish\python\lib\http\client.py", line 956, in send
    self.connect()
  File "c:\users\addy saish\python\lib\http\client.py", line 1392, in connec

RateLimiter swallowed an error after 2 retries. Called with (*('A I Staff Colony S.O,Mumbai,MAHARASHTRA',), **{}).
Traceback (most recent call last):
  File "c:\users\addy saish\python\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "c:\users\addy saish\python\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1016, in _send_output
    self.send(msg)
  File "c:\users\addy saish\python\lib\http\client.py", line 956, in send
    self.connect()
  File "c:\users\addy saish\python\lib\http\client.py", line 1384, in connect
    su

RateLimiter swallowed an error after 2 retries. Called with (*('Airport S.O (Mumbai),Mumbai,MAHARASHTRA',), **{}).
Traceback (most recent call last):
  File "c:\users\addy saish\python\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "c:\users\addy saish\python\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1016, in _send_output
    self.send(msg)
  File "c:\users\addy saish\python\lib\http\client.py", line 956, in send
    self.connect()
  File "c:\users\addy saish\python\lib\http\client.py", line 1392, in connect
    se

RateLimiter swallowed an error after 2 retries. Called with (*('Andheri East S.O,Mumbai,MAHARASHTRA',), **{}).
Traceback (most recent call last):
  File "c:\users\addy saish\python\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "c:\users\addy saish\python\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1016, in _send_output
    self.send(msg)
  File "c:\users\addy saish\python\lib\http\client.py", line 956, in send
    self.connect()
  File "c:\users\addy saish\python\lib\http\client.py", line 1384, in connect
    super(

...

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Uttan S.O,Mumbai,MAHARASHTRA',), **{}).
Traceback (most recent call last):
  File "c:\users\addy saish\python\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "c:\users\addy saish\python\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1016, in _send_output
    self.send(msg)
  File "c:\users\addy saish\python\lib\http\client.py", line 956, in send
    self.connect()
  File "c:\users\addy saish\python\lib\http\client.py", line 1392, in connect
    server_hos

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Vadhavan B.O,Mumbai,MAHARASHTRA',), **{}).
Traceback (most recent call last):
  File "c:\users\addy saish\python\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "c:\users\addy saish\python\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1016, in _send_output
    self.send(msg)
  File "c:\users\addy saish\python\lib\http\client.py", line 956, in send
    self.connect()
  File "c:\users\addy saish\python\lib\http\client.py", line 1384, in connect
    super()

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Vasai Road E S.O,Mumbai,MAHARASHTRA',), **{}).
Traceback (most recent call last):
  File "c:\users\addy saish\python\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "c:\users\addy saish\python\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1016, in _send_output
    self.send(msg)
  File "c:\users\addy saish\python\lib\http\client.py", line 956, in send
    self.connect()
  File "c:\users\addy saish\python\lib\http\client.py", line 1384, in connect
    sup

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Vikramgad S.O,Mumbai,MAHARASHTRA',), **{}).
Traceback (most recent call last):
  File "c:\users\addy saish\python\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "c:\users\addy saish\python\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1016, in _send_output
    self.send(msg)
  File "c:\users\addy saish\python\lib\http\client.py", line 956, in send
    self.connect()
  File "c:\users\addy saish\python\lib\http\client.py", line 1384, in connect
    super(

c:\users\addy saish\python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [47]:
df_mumb.head()

,Neighborhood,pincode,regionname,Address,Coordinates
81395,Antop Hill S.O,400037,Mumbai,"Antop Hill S.O,Mumbai,MAHARASHTRA","(Antop Hill, Mumbai, Mumbai City, Maharashtra,..."
81396,B P T Colony S.O,400037,Mumbai,"B P T Colony S.O,Mumbai,MAHARASHTRA",None
81397,B.P.Lane S.O,400003,Mumbai,"B.P.Lane S.O,Mumbai,MAHARASHTRA",None
81398,BEST STaff Quarters S.O,400012,Mumbai,"BEST STaff Quarters S.O,Mumbai,MAHARASHTRA",None
81399,C G S Colony S.O,400037,Mumbai,"C G S Colony S.O,Mumbai,MAHARASHTRA",None


In [48]:
df_mumb['latitude']=df_mumb['Coordinates'].apply(lambda x:x.latitude if x!=None else None)

c:\users\addy saish\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [49]:
df_mumb['longitude']=df_mumb['Coordinates'].apply(lambda x:x.longitude if x!=None else None)

c:\users\addy saish\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [50]:
df_mumb.head()

,Neighborhood,pincode,regionname,Address,Coordinates,latitude,longitude
81395,Antop Hill S.O,400037,Mumbai,"Antop Hill S.O,Mumbai,MAHARASHTRA","(Antop Hill, Mumbai, Mumbai City, Maharashtra,...",19.020761,72.865256
81396,B P T Colony S.O,400037,Mumbai,"B P T Colony S.O,Mumbai,MAHARASHTRA",None,NaN,NaN
81397,B.P.Lane S.O,400003,Mumbai,"B.P.Lane S.O,Mumbai,MAHARASHTRA",None,NaN,NaN
81398,BEST STaff Quarters S.O,400012,Mumbai,"BEST STaff Quarters S.O,Mumbai,MAHARASHTRA",None,NaN,NaN
81399,C G S Colony S.O,400037,Mumbai,"C G S Colony S.O,Mumbai,MAHARASHTRA",None,NaN,NaN


In [51]:
df_mumb=df_mumb.dropna()

In [52]:
df_mumb.shape

(152, 7)

In [53]:
df_mumb.drop(['Address','Coordinates'],axis=1,inplace=True)

In [54]:
df_del.drop(['Address',"Coordinates"],axis=1,inplace=True)

In [55]:
df_mumb.head()

,Neighborhood,pincode,regionname,latitude,longitude
81395,Antop Hill S.O,400037,Mumbai,19.020761,72.865256
81402,Cotton Exchange S.O,400033,Mumbai,18.987249,72.845762
81403,Dadar Colony S.O,400014,Mumbai,19.027896,72.836529
81408,Kidwai Nagar S.O (Mumbai),400031,Mumbai,19.008436,72.852981
81410,Lal Baug S.O,400012,Mumbai,18.993529,72.842407


In [56]:
df_del.head()

,Neighborhood,pincode,regionname,latitude,longitude
32383,Anand Vihar S.O,110092,Delhi,28.641115,77.312502
32387,Balbir Nagar S.O,110032,Delhi,28.683790,77.290754
32389,Bhola Nath Nagar S.O,110032,Delhi,28.669127,77.285241
32393,Dilshad Garden S.O,110095,Delhi,28.675826,77.321516
32399,Geeta Colony S.O,110031,Delhi,28.650101,77.275921


In [57]:
address='Mumbai'
location=nom.geocode(address)
latitude_mum=location.latitude
longitude_mum=location.longitude
print('{},{}'.format(latitude_mum,longitude_mum))

18.9387711,72.8353355


In [58]:
address='Delhi'
location=nom.geocode(address)
latitude_del=location.latitude
longitude_del=location.longitude
print('{},{}'.format(latitude_del,longitude_del))

28.6517178,77.2219388
